# Optics calculation and matching for a large ring (LHC) - part 1

In [2]:
import xtrack as xt
import os
import numpy as np

In [3]:
#dir_path = os.path.dirname(os.path.realpath(__file__))

# Load a line and build a tracker
line = xt.Line.from_json('lhc_thick_with_knobs.json')
line.build_tracker()

Loading line from dict:   0%|          | 0/14522 [00:00<?, ?it/s]

Done loading line from dict.           


In [4]:
opt = line.match(
    solve=False,
    method='4d', # <- passed to twiss
    vary=[
        xt.VaryList(['kqtf.b1', 'kqtd.b1'], step=1e-8, tag='quad'),
        xt.VaryList(['ksf.b1', 'ksd.b1'], step=1e-4, limits=[-0.1, 0.1], tag='sext'),
    ],
    targets = [
        xt.TargetSet(qx=62.315, qy=60.325, tol=1e-6, tag='tune'),
        xt.TargetSet(dqx=10.0, dqy=12.0, tol=0.01, tag='chrom'),
    ])

In [6]:
opt.solve_homotopy()

Matching: model call n. 6               

Matching: model call n. 13               

Matching: model call n. 20               

Matching: model call n. 27               

Matching: model call n. 34               

Matching: model call n. 41               

Matching: model call n. 48               

Matching: model call n. 55               

Matching: model call n. 62               

Matching: model call n. 69               



In [17]:
opt.log()

Table: 21 rows, 18 cols
iteration       penalty alpha tag           tol_met target_active hit_limits vary_active ...
        0       12.8203    -1               nnnn    yyyy          nnnn       yyyy       
        1    0.00207829     0               yyyy    yyyy          nnnn       yyyy       
        2    0.00207829    -1 Homotopy it 0 yyyy    yyyy          nnnn       yyyy       
        3     0.0033512     0               yyyy    yyyy          nnnn       yyyy       
        4     0.0033512    -1 Homotopy it 1 yyyy    yyyy          nnnn       yyyy       
        5    0.00676554     0               yyyy    yyyy          nnnn       yyyy       
        6    0.00676554    -1 Homotopy it 2 yyyy    yyyy          nnnn       yyyy       
        7   0.000607601     0               yyyy    yyyy          nnnn       yyyy       
        8   0.000607601    -1 Homotopy it 3 yyyy    yyyy          nnnn       yyyy       
        9    0.00254206     0               yyyy    yyyy          nnnn       yyyy 

In [41]:
np.array(opt._log['tag']).shape

(21,)

In [28]:
targets = np.array(opt._log['targets'])[2::2]

In [31]:
targets[:,1]

array([60.32050001, 60.32100001, 60.32150001, 60.32200001, 60.32250001,
       60.32300001, 60.32350001, 60.32400001, 60.32450001, 60.32500001])

In [ ]:
### FOR EXAMPLE
#knobs = np.array(opt._log['knobs']).T
#
#fig, axes = plt.subplots(4)
#knob_keys = list(opt.get_knob_values().keys())
#for i in range(len(knobs)):
#    axes[i].plot(knobs[i])
#    axes[i].set_title(f"Knob {knob_keys[i]}")
#    axes[i].set_xlabel("Iteration")
#    axes[i].set_ylabel("k [T/m]")
#
#plt.tight_layout()
#plt.show()

In [16]:
list(opt.get_knob_values().keys())

['kqtf.b1', 'kqtd.b1', 'ksf.b1', 'ksd.b1']

In [8]:
opt._log['targets']

[array([62.31000003, 60.32      ,  1.9983818 ,  1.98358673]),
 array([62.31050026, 60.32050001,  2.79646548,  2.98520265]),
 array([62.31050026, 60.32050001,  2.79646548,  2.98520265]),
 array([62.31099992, 60.32100001,  3.60205654,  3.98684394]),
 array([62.31099992, 60.32100001,  3.60205654,  3.98684394]),
 array([62.31150033, 60.32150001,  4.39210177,  4.98848489]),
 array([62.31150033, 60.32150001,  4.39210177,  4.98848489]),
 array([62.31200006, 60.32200001,  5.19963611,  5.99012561]),
 array([62.31200006, 60.32200001,  5.19963611,  5.99012561]),
 array([62.31249994, 60.32250001,  6.00173284,  6.99176802]),
 array([62.31249994, 60.32250001,  6.00173284,  6.99176802]),
 array([62.31300008, 60.32300001,  6.79807451,  7.99340824]),
 array([62.31300008, 60.32300001,  6.79807451,  7.99340824]),
 array([62.3135002 , 60.32350001,  7.59923025,  8.99505017]),
 array([62.3135002 , 60.32350001,  7.59923025,  8.99505017]),
 array([62.31400023, 60.32400001,  8.40334325,  9.99669114]),
 array([

In [5]:
merit_function = opt.get_merit_function(return_scalar=True, check_limits=False)
bounds = merit_function.get_x_limits()
x0 = merit_function.get_x()

In [6]:
yt = []
for i in opt.targets:
    yt.append(i.value)
yt = np.array(yt)

In [7]:
def residuals(x):
    x = np.array(x).flatten()
    return merit_function(x) - yt

In [8]:
r0 = residuals(x0)
J0 = merit_function.merit_function.get_jacobian(x0)

In [9]:
get_jacobian = merit_function.merit_function.get_jacobian

In [11]:
b = np.array([[2,3],[1,9]])

In [83]:
import cvxopt
from cvxopt import matrix, solvers

def solve_cp(x0):
    # Convert the initial guess to cvxopt's matrix format
    x0 = matrix(x0)

    def F(x=None, z=None):
        # This function returns the objective and constraints for the solver

        if x is None:  # Initial point
            return 0, x0

        # Convert x to a numpy array
        x_np = np.array(x)

        f = merit_function(x_np)  # Objective function
        Df = np.sum(merit_function.merit_function.get_jacobian(x_np), axis=1)

        if z is None:
            return f, matrix(Df)

        # With Lagrange multipliers (z), return Hessian approximation
        H = z[0] * (Df.T @ Df)
        return f, matrix(Df), matrix(H)
    sol = solvers.cp(F)
    return sol

In [84]:
solution = solve_cp(x0)

     pcost       dcost       gap    pres   dres


TypeError: length of x is too small

In [ ]:
xx = matrix(1.0, (4,1))

In [ ]:
ff = -sum(log(xx))

In [ ]:
Df = -(xx**-1).T

In [ ]:
print(Df)

In [94]:
from cvxopt import solvers, matrix, spdiag, log

def acent(A, b):
    m, n = A.size
    def F(x=None, z=None):
        if x is None: return 0, matrix(1.0, (n,1))
        if min(x) <= 0.0: return None
        f = -sum(log(x))
        Df = -(x**-1).T
        if z is None: return f, Df
        H = spdiag(z[0] * x**-2)
        return f, Df, H
    return solvers.cp(F, A=A, b=b)

In [101]:
A = matrix(np.eye(4)) * 1000

In [104]:
A = matrix(np.random.rand(4, 4) + np.eye(4) * 1000)
print(A.size)

(4, 4)


In [106]:
b = matrix(np.random.rand(4))
print(b)

[ 4.17e-01]
[ 4.48e-01]
[ 5.15e-01]
[ 3.51e-01]



In [107]:
d = acent(A, b)

     pcost       dcost       gap    pres   dres
 0:  0.0000e+00  0.0000e+00  1e+00  1e+00  1e+00
 1:  3.9583e+00  1.8333e+01  1e-02  1e-02  9e+01
 2:  2.1907e+01  3.0257e+01  1e-04  4e-03  2e+03
 3:  3.0853e+01  3.1028e+01  1e-06  8e-05  1e+02
 4:  3.1027e+01  3.1028e+01  1e-08  8e-07  1e+00
 5:  3.1028e+01  3.1028e+01  1e-10  8e-09  1e-02
 6:  3.1028e+01  3.1028e+01  1e-12  8e-11  1e-04
 7:  3.1028e+01  3.1028e+01  1e-14  8e-13  1e-06
 8:  3.1028e+01  3.1028e+01  1e-16  8e-15  1e-08
Optimal solution found.


In [108]:
d

{'status': 'optimal',
 'x': <4x1 matrix, tc='d'>,
 'y': <4x1 matrix, tc='d'>,
 'znl': <0x1 matrix, tc='d'>,
 'zl': <0x1 matrix, tc='d'>,
 'snl': <0x1 matrix, tc='d'>,
 'sl': <0x1 matrix, tc='d'>,
 'gap': 1.0000000000000077e-16,
 'relative gap': 3.2228508396975407e-18,
 'primal objective': 31.028429478706155,
 'dual objective': 31.028429478723755,
 'primal slack': 1.0000000000000069e-16,
 'dual slack': 1.0000000000000009,
 'primal infeasibility': 8.316827601243476e-15,
 'dual infeasibility': 1.1144010556622795e-08}

In [112]:
print(d['y'])

[ 2.40e+00]
[ 2.23e+00]
[ 1.94e+00]
[ 2.85e+00]

